In [70]:
import cv2
import numpy as np
import pandas as pd
# print version number
import os

In [71]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')  


# Import faces to learn from 

In [72]:
import face_recognition
image = face_recognition.load_image_file("../data/raw/SS/IMG_0684.jpeg")
face_ss = face_recognition.face_encodings(image)[0]
image = face_recognition.load_image_file("../data/raw/DG/IMG_1135.jpeg")
face_dg = face_recognition.face_encodings(image)[0]

# Create audio files

In [73]:
from gtts import gTTS 
text_ss = 'Welcome Sunanda!'
text_dg = 'Welcome Danilo!'
text_stranger = "Welcome Stranger!"
language = 'en'
sound_ss = gTTS(text=text_ss, lang=language, slow=False)
sound_ss.save("../data/raw/SS/ss.mp3") 
sound_dg = gTTS(text=text_dg, lang=language, slow=False)
sound_dg.save("../data/raw/DG/dg.mp3") 
sound_st = gTTS(text=text_stranger, lang=language, slow=False)
sound_st.save("../data/raw/stranger.mp3") 


In [74]:
known_face_encodings = [
    face_ss, face_dg
]
known_face_names = [
    "Sunanda","Danilo"
]
known_faces_audio = [
    "../data/raw/SS/ss.mp3","../data/raw/DG/dg.mp3"
]

In [75]:
face_locations = []
face_encodings = []
face_names = []
face_sounds = []
process_this_frame = True

In [ ]:
cap = cv2.VideoCapture(0) 
missing_cnt ={'Sunanda':-1,
             'Danilo':-1,
             'Stranger':-1}
while True:
    ret, frame = cap.read()

    # Resize frame of video to 1/4 size for faster face recognition processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
    rgb_small_frame = small_frame[:, :, ::-1]
    
    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
    face_names = []

    for face_encoding in face_encodings:
        name = "Stranger"
        sound = "../data/raw/stranger.mp3"
        
        # Check if we know this face
        
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        # choose face with least diff
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            sound = known_faces_audio[best_match_index]

        if missing_cnt[name]==-1 or missing_cnt[name]>20:
            #if we are seeing this person for the first time or after 20 seconds of absence
            #Welcome them :)
            
            playsound(sound)
            if missing_cnt[name]>20:
                missing_cnt[name]=1
            else:
                missing_cnt[name]+=1
        
        face_names.append(name)
        face_sounds.append(sound)

    #update time of absence if a person came and left
    for n in [x for x in missing_cnt.keys() if x not in face_names and missing_cnt[x]!=-1]:
        missing_cnt.update({n: missing_cnt[n]+1})

    #to avoid unnecessary overflow, after 100 frames, reset to -1
    for k,v in missing_cnt.items():
        if v>100:
            missing_cnt[k]=-1


    # Display the box
    for (top, right, bottom, left), name, sound, sound_flag in zip(face_locations, face_names, face_sounds, sound_flags):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        cv2.rectangle(frame, (left, top), (right, bottom), (255, 255, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (255, 255, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_SCRIPT_COMPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (0, 0, 0), 1)
        
        
    # Display the resulting image
    cv2.imshow('Hello faces!', frame)

    # quit by pressing q
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
cap.release()
cv2.destroyAllWindows()